<a href="https://colab.research.google.com/github/anindabitm/CGIAR-Zindi/blob/master/Tree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
!ls '/content/drive/My Drive/cgiar-wheat/'

In [3]:
!cp '/content/drive/My Drive/cgiar-wheat/Images.zip' .
!unzip -q Images.zip
!rm Images.zip

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image
import torch 
import torch.nn as nn
from PIL import Image

In [53]:
train = pd.read_csv('/content/drive/My Drive/cgiar-wheat/Train.csv')
train['images'] = train['UID'].apply(lambda x: x + ".jpeg")
train.head()

,UID,growth_stage,label_quality,images
0,nwuCTQKz,4,2,nwuCTQKz.jpeg
1,NVGUOzWu,1,1,NVGUOzWu.jpeg
2,1IYVvHwA,1,1,1IYVvHwA.jpeg
3,8OEdWwmU,1,1,8OEdWwmU.jpeg
4,oc5iO04j,1,1,oc5iO04j.jpeg


In [54]:
train.shape

(10695, 4)

In [55]:
test = pd.read_csv('/content/drive/My Drive/cgiar-wheat/SampleSubmission.csv')
test['images'] = test['UID'].apply(lambda x: x + ".jpeg")
test.head()

,UID,growth_stage,images
0,F3LbWkZq,0,F3LbWkZq.jpeg
1,6P4wLVzI,0,6P4wLVzI.jpeg
2,Tn6HIrdS,0,Tn6HIrdS.jpeg
3,EYM5R1TK,0,EYM5R1TK.jpeg
4,fkcevlA0,0,fkcevlA0.jpeg


In [56]:
train_hiq = train.loc[train.label_quality==2,:].reset_index(drop=True)
train_loq = train.loc[train.label_quality==1,:].reset_index(drop=True)
train_hiq.shape,train_loq.shape

((2856, 4), (7839, 4))

In [57]:
img_arr1 = np.array([
            np.array(((Image.open('Images/' + filename)).convert("RGB")).resize((56, 56),resample=Image.BILINEAR))
               for filename in train_hiq.images.values])

img_arr2 = np.array([
            np.array(((Image.open('Images/' + filename)).convert("RGB")).resize((56, 56),resample=Image.BILINEAR))
               for filename in train_loq.images.values])

In [58]:
print(img_arr2[:,:,:,0].reshape(7839,-1,).shape)
print(img_arr2[:,:,:,1].reshape(7839,-1,).shape)
print(img_arr2[:,:,:,2].reshape(7839,-1,).shape)

(7839, 3136)
(7839, 3136)
(7839, 3136)


In [59]:
xtrain = np.hstack((img_arr1[:,:,:,0].reshape(2856,-1,),img_arr1[:,:,:,1].reshape(2856,-1,),img_arr1[:,:,:,2].reshape(2856,-1,)))
xtest = np.hstack((img_arr2[:,:,:,0].reshape(7839,-1,),img_arr2[:,:,:,1].reshape(7839,-1,),img_arr2[:,:,:,2].reshape(7839,-1,)))

In [60]:
xtrain.shape,xtest.shape

((2856, 9408), (7839, 9408))

In [61]:
ytrain = train_hiq.growth_stage.values

In [62]:
ytrain.shape

(2856,)

In [64]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3,shuffle=True,random_state=42)
fold = 0
preds = np.zeros(shape=(7839,))
for train_index, test_index in skf.split(xtrain, ytrain):
    fold+= 1
    print('In fold',fold)
    print("TRAIN LENGTH:", len(train_index), "VALIDATION LENGTH:", len(test_index))
    train_x = xtrain[train_index]
    train_y = ytrain[train_index]
    valid_x = xtrain[test_index]
    valid_y = ytrain[test_index]
    model = lgb.LGBMRegressor(n_estimators=300)
    model.fit(train_x,train_y,eval_set=(valid_x,valid_y),early_stopping_rounds=10,verbose=20)
    preds += model.predict(xtest)

preds = preds/3

In fold 1
TRAIN LENGTH: 1904 VALIDATION LENGTH: 952
Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l2: 1.15011
[40]	valid_0's l2: 1.07695
[60]	valid_0's l2: 1.05897
[80]	valid_0's l2: 1.04998
[100]	valid_0's l2: 1.04636
[120]	valid_0's l2: 1.04342
[140]	valid_0's l2: 1.04237
[160]	valid_0's l2: 1.04179
[180]	valid_0's l2: 1.04139
Early stopping, best iteration is:
[177]	valid_0's l2: 1.04132
In fold 2
TRAIN LENGTH: 1904 VALIDATION LENGTH: 952
Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l2: 1.08873
[40]	valid_0's l2: 1.01778
[60]	valid_0's l2: 1.00445
[80]	valid_0's l2: 0.994747
Early stopping, best iteration is:
[83]	valid_0's l2: 0.993855
In fold 3
TRAIN LENGTH: 1904 VALIDATION LENGTH: 952
Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l2: 1.09932
[40]	valid_0's l2: 1.0025
[60]	valid_0's l2: 0.976977
[80]	valid_0's l2: 0.966065
[100]	valid_0's l2: 0.959579
Early stopping, best iteration is:
[1

In [65]:
train_loq['growth_stage'] = preds
train_loq.head()

,UID,growth_stage,label_quality,images
0,NVGUOzWu,6.409744,1,NVGUOzWu.jpeg
1,1IYVvHwA,3.022923,1,1IYVvHwA.jpeg
2,8OEdWwmU,3.526897,1,8OEdWwmU.jpeg
3,oc5iO04j,4.855008,1,oc5iO04j.jpeg
4,A60rOyP9,2.884030,1,A60rOyP9.jpeg


In [66]:
train = pd.concat([train_hiq,train_loq]).reset_index(drop=True)
train = train.sample(frac=1).reset_index(drop=True)
train.shape

(10695, 4)

In [67]:
img_train = np.array([
            np.array(((Image.open('Images/' + filename)).convert("RGB")).resize((56, 56),resample=Image.BILINEAR))
               for filename in train.images.values])

img_test = np.array([
            np.array(((Image.open('Images/' + filename)).convert("RGB")).resize((56, 56),resample=Image.BILINEAR))
               for filename in test.images.values])

In [68]:
test.shape

(3558, 3)

In [69]:
xtrain = np.hstack((img_train[:,:,:,0].reshape(10695,-1,),img_train[:,:,:,1].reshape(10695,-1,),img_train[:,:,:,2].reshape(10695,-1,)))
xtest = np.hstack((img_test[:,:,:,0].reshape(3558,-1,),img_test[:,:,:,1].reshape(3558,-1,),img_test[:,:,:,2].reshape(3558,-1,)))
ytrain = train['growth_stage'].values
xtrain.shape,ytrain.shape,xtest.shape

((10695, 9408), (10695,), (3558, 9408))

In [70]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,StratifiedKFold
kf = KFold(n_splits=5,shuffle=True,random_state=42)
fold = 0
preds = np.zeros(shape=(3558,))
for train_index, test_index in kf.split(xtrain):
    fold+= 1
    print('In fold',fold)
    print("TRAIN LENGTH:", len(train_index), "VALIDATION LENGTH:", len(test_index))
    train_x = xtrain[train_index]
    train_y = ytrain[train_index]
    valid_x = xtrain[test_index]
    valid_y = ytrain[test_index]
    model = lgb.LGBMRegressor(n_estimators=300)
    model.fit(train_x,train_y,eval_set=(valid_x,valid_y),early_stopping_rounds=10,verbose=20)
    preds += model.predict(xtest)

preds = preds/5

In fold 1
TRAIN LENGTH: 8556 VALIDATION LENGTH: 2139
Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l2: 0.372256
[40]	valid_0's l2: 0.315017
[60]	valid_0's l2: 0.293943
[80]	valid_0's l2: 0.284506
[100]	valid_0's l2: 0.278931
[120]	valid_0's l2: 0.275702
[140]	valid_0's l2: 0.274444
[160]	valid_0's l2: 0.272872
[180]	valid_0's l2: 0.271235
[200]	valid_0's l2: 0.269708
[220]	valid_0's l2: 0.269215
[240]	valid_0's l2: 0.268545
[260]	valid_0's l2: 0.268277
[280]	valid_0's l2: 0.267857
[300]	valid_0's l2: 0.267247
Did not meet early stopping. Best iteration is:
[300]	valid_0's l2: 0.267247
In fold 2
TRAIN LENGTH: 8556 VALIDATION LENGTH: 2139
Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l2: 0.367935
[40]	valid_0's l2: 0.303894
[60]	valid_0's l2: 0.283775
[80]	valid_0's l2: 0.275875
[100]	valid_0's l2: 0.270303
[120]	valid_0's l2: 0.267863
[140]	valid_0's l2: 0.265545
[160]	valid_0's l2: 0.263852
[180]	valid_0's l2: 0.263053
[200]

In [71]:
test['growth_stage'] = preds
test[['UID','growth_stage']].to_csv('/content/drive/My Drive/cgiar-wheat/mlmodel.csv',index=False)